In [2]:
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
df_data = pd.read_csv('parsed_file_dida2_orphanet.csv', index_col=0)
df_data.head()

,DEO1,DEO2,Rec1,EssA,DEO3,DEO4,RecB,EssB,Distance,Pathway,DE,Orphanet,Name,Combination
DIDAid,,,,,,,,,,,,,,
dd001,0.99,0.0,0.34,0.000,1.00,0.0,0.30,0.392,1.11,1,CO,768,Familial long QT syndrome,KCNH2/KCNQ1
dd002,0.39,0.0,0.16,0.392,1.00,0.0,0.51,0.392,1.25,1,TD,87884,Non-syndromic genetic deafness,GJB2/GJB3
dd003,0.40,0.0,0.16,0.392,1.00,0.0,0.51,0.392,1.25,1,TD,87884,Non-syndromic genetic deafness,GJB2/GJB3
dd004,0.40,0.0,0.16,0.392,1.00,0.0,0.51,0.392,1.25,1,TD,87884,Non-syndromic genetic deafness,GJB2/GJB3
dd005,0.12,0.0,0.20,0.392,0.21,0.0,0.44,0.000,1.25,0,TD,87884,Non-syndromic genetic deafness,FOXI1/SLC26A4


In [4]:
X, y = array(df_data)[:,:10].astype(float), array(df_data.replace('UK', 0).replace('TD', 1).replace('CO', -1))[:,10].astype(int)

In [137]:
class GraphEnergyClassifier:
    
    def __init__(self, similarity='default', l1='default', l2='default', c1='default', c2='default', eps='default'):
        
        # Weights between vertices, dict of (i, j)
        self.weights = {}
        
        # Convex loss function parameters
        self.similarity, self.l1, self.l2, self.c1, self.c2, self.eps = (
            lambda x_1, x_2: 1 / (1 + sum(abs(x_1 - x_2))) if similarity == 'default' else similarity,
            1 if l1 == 'default' else l1,
            1 if l2 == 'default' else l2,
            (lambda i, j: abs(i - j)**2) if c1 == 'default' else c1,
            (lambda i, j: abs(i - j))    if c2 == 'default' else c2,
            1e-9 if eps == 'default' else eps
        )
        
        # Learning values
        self.X_fit, self.y_fit = None, None
        self.predictions = None
        
        # Amount of labeled/unlabeled data
        self.l, self.u = 0, 0
        
    def __len__(self):
        """ __len(self)__ -> number of samples """
        assert self.predictions is not None, "Model must be fit."
        
        return len(self.predictions)
        
    def _weight_(self, i, j):
        """ _weight_(int, int) -> float """
        assert self.X_fit is not None and self.y_fit is not None, "Model must be fit."
        
        i, j = min(i, j), max(i, j)
        if (i, j) not in self.weights:
            self.weights[(i, j)] = self.similarity( self.X_fit[i], self.y_fit[j] )
        return self.weights[(i, j)]
    
    def _label_(self, i):
        """ _label_(i) -> {-1, 0, 1} """
        assert self.y_fit is not None, "Model must be fit."
        assert i < len(self.y_fit), "Index out of range : " + str(i)
        
        return self.y_fit[i]
    
    def _energy_(self, f):
        """ _energy_(self, f) -> energy necessary to affectation f """
        return (
            1 / self.l * sum(
                self.c1(self._label_(i), f[i])
                for i in range(self.l)
            ) +
            self.l1 * sum(
               f**2 
            ) +
            self.l2 * sum(
                sum(
                    self.c2(f[i], f[j])*self._weight_(i, j)
                    for j in range(i)
                ) for i in range(self.l + self.u)
            )
        )
    
    def _gradient_(self, f):
        """ _gradient_(self, f) -> partial derivatives of energy at point f """
        return array([
            (self._energy_(f + array([j == i for j in range(self.l + self.u)])*self.eps) -
            self._energy_(f - array([j == i for j in range(self.l + self.u)])*self.eps)) / (2 * self.eps)
            for i in range(self.l + self.u)
        ])
    
    def fit(self, X, y, learning_rate='default', max_iter='default', starting_point='default'):
        assert len(X) == len(y), "Design matrix and class vector must have same length."
        
        # Data loading, putting unlabeled at the end
        labeled_selector = y != 0
        self.X_fit = concatenate( (X[labeled_selector], X[labeled_selector == False]) )
        self.y_fit = concatenate( (y[labeled_selector], y[labeled_selector == False]) )
        
        # Labeled and unlabeled data size
        self.l, self.u = sum(labeled_selector), len(labeled_selector) - sum(labeled_selector)
        
        # Gradient descent parameters
        # x : initial value of x
        learning_rate, max_iter, x = (
            1e-4 if learning_rate == 'default' else learning_rate,
            10 if max_iter == 'default' else max_iter,
            array([0 for _ in range(len(self.y_fit))]).astype(float) if starting_point == 'default' else starting_point
        )
        
        print('#'*20)
        print('Beginning of fitting.')
        
        # Gradient descent loop
        for i in range(max_iter):
            g = self._gradient_(x)
            if sum(abs(g**2)) < self.eps:
                break
            print("Round :", i, "Step : ", sum(abs(g*learning_rate)))
            x -= g*learning_rate
        return x

In [138]:
g = GraphEnergyClassifier(l1=0.01, l2=0.01)
x = g.fit(X, y)

####################
Beginning of fitting.
Round : 0 Step :  0.000199999838912
Round : 1 Step :  0.00285061988459
Round : 2 Step :  0.00325062885498
Round : 3 Step :  0.00333037584177
Round : 4 Step :  0.0036035632589
Round : 5 Step :  0.00332556734373
Round : 6 Step :  0.00371021736667
Round : 7 Step :  0.00333289942311
Round : 8 Step :  0.00368961918795
Round : 9 Step :  0.00335672154517


In [140]:
selector = g.y_fit != 0
sum(g.y_fit[selector] == ((x[selector] < 0) * 2 - 1))/len(x[selector])

0.67878787878787883